<a href="https://colab.research.google.com/github/friedelj/AAI-510-TEAM-03/blob/main/JFriedel_CV_Assignment5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Joseph Friedel**      Computer Vision      **Assignment 5**    11_25_24

**Part 1- Video read/write**




In [ ]:
from google.colab.patches import cv2_imshow
import cv2
import numpy as np
from google.colab import files

In [ ]:
# Open the video file
cap = cv2.VideoCapture('/content/Bangkok.mp4')

# Retrieve metadata
video_fps = cap.get(cv2.CAP_PROP_FPS)  # Frames per second
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # Total number of frames
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # Width of the video
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # Height of the video

# Print the video metadata
print(f"Frame Per second: {video_fps} \nTotal Frames: {total_frames} \nHeight: {height} \nWidth: {width}")

# Release the video capture object
cap.release()

Frame Per second: 25.0 
Total Frames: 650 
Height: 720 
Width: 1280


In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload the video file manually

video_path = "Bangkok.mp4"  # Use the uploaded file name

Saving Bangkok.mp4 to Bangkok (1).mp4


In [ ]:
#b) Play the file- method 1
# Check if camera opened successfully
if (cap.isOpened() == False):
  print("Error opening video stream or file")

while(True):
  ret, frame = cap.read()
  if ret == True:
    # Display the resulting frame
    cv2_imshow(frame)
    # Press Q on keyboard to stop recording
    if cv2.waitKey(1) & 0xFF == ord('q'):
      break
  # Break the loop
  else:
    break
# When everything done, release the video capture and video write objects
cap.release()
# Closes all the frames
cv2.destroyAllWindows()


Error opening video stream or file


In [ ]:
# Upload file to Colab
uploaded = files.upload()  # Upload the video file manually

# Set the video file path (ensure the name matches the uploaded file)
video_path = "Bangkok.mp4"

# Create a VideoCapture object
cap = cv2.VideoCapture(video_path)

# Check if the video opened successfully
if not cap.isOpened():
    print("Error opening video stream or file")
else:
    while True:
        ret, frame = cap.read()
        if ret:
            # Display the resulting frame
            cv2_imshow(frame)
        else:
            break

# Release the video capture object
cap.release()
cv2.destroyAllWindows()

In [ ]:
#Play the file- method 2
from IPython.display import HTML
from base64 import b64encode
video_path = "/content/Bangkok.mp4"

mp4 = open(video_path,'rb').read()
decoded_vid = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f'<video width=400 controls><source src={decoded_vid} type="video/mp4"></video>')

In [ ]:
# Define the paths
video_path = "/content/Bangkok.mp4"
out_video_path = "/content/processed_Bangkok.mp4"

# Open the input video
cap = cv2.VideoCapture(video_path)

# Retrieve metadata
video_fps = cap.get(cv2.CAP_PROP_FPS)  # Frames per second
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # Total number of frames
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # Width of the video
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # Height of the video

# Print the metadata
print(f"Frame Per second: {video_fps} \nTotal Frames: {total_frames} \nHeight: {height} \nWidth: {width}")

# Define the VideoWriter
fourcc = cv2.VideoWriter_fourcc(*'X264')  # Codec
writer = cv2.VideoWriter(out_video_path, fourcc, video_fps, (width, height))

# Process frames (optional example: copy input frames to the output)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    writer.write(frame)  # Write the frame to the output video

# Release resources
cap.release()
writer.release()

Frame Per second: 25.0 
Total Frames: 650 
Height: 720 
Width: 1280


In [ ]:
# Creating a VideoCapture object to read the video
video_path = "Bangkok.mp4"  # Address of the file
cap = cv2.VideoCapture(video_path)  # Open the path in OpenCV

# Check if the video opened successfully
if not cap.isOpened():
    print("Error opening video stream or file")
else:
    # Loop until the end of the video or until 3 frames are displayed
    frame_count = 0  # To keep track of the number of frames displayed
    while cap.isOpened():
        # Capture frame-by-frame
        ret, frame = cap.read()

        if ret:  # If a frame is successfully captured
            # Convert the frame to grayscale
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Apply adaptive thresholding to create a sketchy image
            Thresh = cv2.adaptiveThreshold(
                gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2
            )

            # Display the sketchy frame
            cv2_imshow(Thresh)

            frame_count += 1
            if frame_count == 3:  # Stop after displaying 3 frames
                break
        else:
            break

    # Release the video capture object
    cap.release()
    # Close all OpenCV windows
    cv2.destroyAllWindows()

**Part 2- Pose estimation in video**

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
#from tensorflow_docs.vis import embed
import numpy as np
import cv2

# Import matplotlib libraries
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib.patches as patches

# Some modules to display an animation using imageio.
import imageio
import requests
from IPython.display import HTML, Image, display

In [ ]:
model_name = "movenet_lightning"

if "tflite" in model_name:
  if "movenet_lightning_f16" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/lightning/tflite/float16/4?lite-format=tflite
    input_size = 192
  elif "movenet_thunder_f16" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/thunder/tflite/float16/4?lite-format=tflite
    input_size = 256
  elif "movenet_lightning_int8" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/lightning/tflite/int8/4?lite-format=tflite
    input_size = 192
  elif "movenet_thunder_int8" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/thunder/tflite/int8/4?lite-format=tflite
    input_size = 256
  else:
    raise ValueError("Unsupported model name: %s" % model_name)

  # Initialize the TFLite interpreter
  interpreter = tf.lite.Interpreter(model_path="model.tflite")
  interpreter.allocate_tensors()

  def movenet(input_image):
    """Runs detection on an input image.

    Args:
      input_image: A [1, height, width, 3] tensor represents the input image
        pixels. Note that the height/width should already be resized and match the
        expected input resolution of the model before passing into this function.

    Returns:
      A [1, 1, 17, 3] float numpy array representing the predicted keypoint
      coordinates and scores.
    """
    # TF Lite format expects tensor type of uint8.
    input_image = tf.cast(input_image, dtype=tf.uint8)
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    interpreter.set_tensor(input_details[0]['index'], input_image.numpy())
    # Invoke inference.
    interpreter.invoke()
    # Get the model prediction.
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    return keypoints_with_scores

else:
  if "movenet_lightning" in model_name:
    module = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
    input_size = 192
  elif "movenet_thunder" in model_name:
    module = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")
    input_size = 256
  else:
    raise ValueError("Unsupported model name: %s" % model_name)

  def movenet(input_image):
    """Runs detection on an input image.

    Args:
      input_image: A [1, height, width, 3] tensor represents the input image
        pixels. Note that the height/width should already be resized and match the
        expected input resolution of the model before passing into this function.

    Returns:
      A [1, 1, 17, 3] float numpy array representing the predicted keypoint
      coordinates and scores.
    """
    model = module.signatures['serving_default']

    # SavedModel format expects tensor type of int32.
    input_image = tf.cast(input_image, dtype=tf.int32)
    # Run model inference.
    outputs = model(input_image)
    # Output is a [1, 1, 17, 3] tensor.
    keypoints_with_scores = outputs['output_0'].numpy()
    return keypoints_with_scores

In [ ]:
#Read the required gif file from https://github.com/tensorflow/tfjs-models/raw/master/pose-detection/assets/dance_input.gif
#Put code here
# URL of the GIF
gif_url = "https://github.com/tensorflow/tfjs-models/raw/master/pose-detection/assets/dance_input.gif"

# Download the GIF
response = requests.get(gif_url)
if response.status_code == 200:
    with open("dance_input.gif", "wb") as f:
        f.write(response.content)
else:
    print("Failed to download GIF")

# Read the GIF using imageio
gif_reader = imageio.mimread("dance_input.gif")

# Display the first frame of the GIF as an example
display(Image(data=open("dance_input.gif", "rb").read(), format="gif"))

In [ ]:
# Load the input image.
image_path = 'dance_input.gif'
image = tf.io.read_file(image_path)
image = tf.image.decode_gif(image)


In [ ]:
#All the required code are put in this cell you don't need to modify anything in this cell
# Confidence score to determine whether a keypoint prediction is reliable.
MIN_CROP_KEYPOINT_SCORE = 0.2

KEYPOINT_DICT = {
    'nose': 0,
    'left_eye': 1,
    'right_eye': 2,
    'left_ear': 3,
    'right_ear': 4,
    'left_shoulder': 5,
    'right_shoulder': 6,
    'left_elbow': 7,
    'right_elbow': 8,
    'left_wrist': 9,
    'right_wrist': 10,
    'left_hip': 11,
    'right_hip': 12,
    'left_knee': 13,
    'right_knee': 14,
    'left_ankle': 15,
    'right_ankle': 16
}

KEYPOINT_EDGE_INDS_TO_COLOR = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}
def init_crop_region(image_height, image_width):
  """Defines the default crop region.

  The function provides the initial crop region (pads the full image from both
  sides to make it a square image) when the algorithm cannot reliably determine
  the crop region from the previous frame.
  """
  if image_width > image_height:
    box_height = image_width / image_height
    box_width = 1.0
    y_min = (image_height / 2 - image_width / 2) / image_height
    x_min = 0.0
  else:
    box_height = 1.0
    box_width = image_height / image_width
    y_min = 0.0
    x_min = (image_width / 2 - image_height / 2) / image_width

  return {
    'y_min': y_min,
    'x_min': x_min,
    'y_max': y_min + box_height,
    'x_max': x_min + box_width,
    'height': box_height,
    'width': box_width
  }

def torso_visible(keypoints):
  """Checks whether there are enough torso keypoints.

  This function checks whether the model is confident at predicting one of the
  shoulders/hips which is required to determine a good crop region.
  """
  return ((keypoints[0, 0, KEYPOINT_DICT['left_hip'], 2] >
           MIN_CROP_KEYPOINT_SCORE or
          keypoints[0, 0, KEYPOINT_DICT['right_hip'], 2] >
           MIN_CROP_KEYPOINT_SCORE) and
          (keypoints[0, 0, KEYPOINT_DICT['left_shoulder'], 2] >
           MIN_CROP_KEYPOINT_SCORE or
          keypoints[0, 0, KEYPOINT_DICT['right_shoulder'], 2] >
           MIN_CROP_KEYPOINT_SCORE))


def _keypoints_and_edges_for_display(keypoints_with_scores,
                                     height,
                                     width,
                                     keypoint_threshold=0.11):
  """Returns high confidence keypoints and edges for visualization.

  Args:
    keypoints_with_scores: A numpy array with shape [1, 1, 17, 3] representing
      the keypoint coordinates and scores returned from the MoveNet model.
    height: height of the image in pixels.
    width: width of the image in pixels.
    keypoint_threshold: minimum confidence score for a keypoint to be
      visualized.

  Returns:
    A (keypoints_xy, edges_xy, edge_colors) containing:
      * the coordinates of all keypoints of all detected entities;
      * the coordinates of all skeleton edges of all detected entities;
      * the colors in which the edges should be plotted.
  """
  keypoints_all = []
  keypoint_edges_all = []
  edge_colors = []
  num_instances, _, _, _ = keypoints_with_scores.shape
  for idx in range(num_instances):
    kpts_x = keypoints_with_scores[0, idx, :, 1]
    kpts_y = keypoints_with_scores[0, idx, :, 0]
    kpts_scores = keypoints_with_scores[0, idx, :, 2]
    kpts_absolute_xy = np.stack(
        [width * np.array(kpts_x), height * np.array(kpts_y)], axis=-1)
    kpts_above_thresh_absolute = kpts_absolute_xy[
        kpts_scores > keypoint_threshold, :]
    keypoints_all.append(kpts_above_thresh_absolute)

    for edge_pair, color in KEYPOINT_EDGE_INDS_TO_COLOR.items():
      if (kpts_scores[edge_pair[0]] > keypoint_threshold and
          kpts_scores[edge_pair[1]] > keypoint_threshold):
        x_start = kpts_absolute_xy[edge_pair[0], 0]
        y_start = kpts_absolute_xy[edge_pair[0], 1]
        x_end = kpts_absolute_xy[edge_pair[1], 0]
        y_end = kpts_absolute_xy[edge_pair[1], 1]
        line_seg = np.array([[x_start, y_start], [x_end, y_end]])
        keypoint_edges_all.append(line_seg)
        edge_colors.append(color)
  if keypoints_all:
    keypoints_xy = np.concatenate(keypoints_all, axis=0)
  else:
    keypoints_xy = np.zeros((0, 17, 2))

  if keypoint_edges_all:
    edges_xy = np.stack(keypoint_edges_all, axis=0)
  else:
    edges_xy = np.zeros((0, 2, 2))
  return keypoints_xy, edges_xy, edge_colors


def progress(value, max=100):
  return HTML("""
      <progress
          value='{value}'
          max='{max}',
          style='width: 100%'
      >
          {value}
      </progress>
  """.format(value=value, max=max))

def crop_and_resize(image, crop_region, crop_size):
  """Crops and resize the image to prepare for the model input."""
  boxes=[[crop_region['y_min'], crop_region['x_min'],
          crop_region['y_max'], crop_region['x_max']]]
  output_image = tf.image.crop_and_resize(
      image, box_indices=[0], boxes=boxes, crop_size=crop_size)
  return output_image

def run_inference(movenet, image, crop_region, crop_size):
  """Runs model inferece on the cropped region.

  The function runs the model inference on the cropped region and updates the
  model output to the original image coordinate system.
  """
  image_height, image_width, _ = image.shape
  input_image = crop_and_resize(
    tf.expand_dims(image, axis=0), crop_region, crop_size=crop_size)
  # Run model inference.
  keypoints_with_scores = movenet(input_image)
  # Update the coordinates.
  for idx in range(17):
    keypoints_with_scores[0, 0, idx, 0] = (
        crop_region['y_min'] * image_height +
        crop_region['height'] * image_height *
        keypoints_with_scores[0, 0, idx, 0]) / image_height
    keypoints_with_scores[0, 0, idx, 1] = (
        crop_region['x_min'] * image_width +
        crop_region['width'] * image_width *
        keypoints_with_scores[0, 0, idx, 1]) / image_width
  return keypoints_with_scores

def draw_prediction_on_image(
    image, keypoints_with_scores, crop_region=None, close_figure=False,
    output_image_height=None):
  """Draws the keypoint predictions on image.

  Args:
    image: A numpy array with shape [height, width, channel] representing the
      pixel values of the input image.
    keypoints_with_scores: A numpy array with shape [1, 1, 17, 3] representing
      the keypoint coordinates and scores returned from the MoveNet model.
    crop_region: A dictionary that defines the coordinates of the bounding box
      of the crop region in normalized coordinates (see the init_crop_region
      function below for more detail). If provided, this function will also
      draw the bounding box on the image.
    output_image_height: An integer indicating the height of the output image.
      Note that the image aspect ratio will be the same as the input image.

  Returns:
    A numpy array with shape [out_height, out_width, channel] representing the
    image overlaid with keypoint predictions.
  """
  height, width, channel = image.shape
  aspect_ratio = float(width) / height
  fig, ax = plt.subplots(figsize=(12 * aspect_ratio, 12))
  # To remove the huge white borders
  fig.tight_layout(pad=0)
  ax.margins(0)
  ax.set_yticklabels([])
  ax.set_xticklabels([])
  plt.axis('off')

  im = ax.imshow(image)
  line_segments = LineCollection([], linewidths=(4), linestyle='solid')
  ax.add_collection(line_segments)
  # Turn off tick labels
  scat = ax.scatter([], [], s=60, color='#FF1493', zorder=3)

  (keypoint_locs, keypoint_edges,
   edge_colors) = _keypoints_and_edges_for_display(
       keypoints_with_scores, height, width)

  line_segments.set_segments(keypoint_edges)
  line_segments.set_color(edge_colors)
  if keypoint_edges.shape[0]:
    line_segments.set_segments(keypoint_edges)
    line_segments.set_color(edge_colors)
  if keypoint_locs.shape[0]:
    scat.set_offsets(keypoint_locs)

  if crop_region is not None:
    xmin = max(crop_region['x_min'] * width, 0.0)
    ymin = max(crop_region['y_min'] * height, 0.0)
    rec_width = min(crop_region['x_max'], 0.99) * width - xmin
    rec_height = min(crop_region['y_max'], 0.99) * height - ymin
    rect = patches.Rectangle(
        (xmin,ymin),rec_width,rec_height,
        linewidth=1,edgecolor='b',facecolor='none')
    ax.add_patch(rect)

  fig.canvas.draw()
  image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
  image_from_plot = image_from_plot.reshape(
      fig.canvas.get_width_height()[::-1] + (3,))
  plt.close(fig)
  if output_image_height is not None:
    output_image_width = int(output_image_height / height * width)
    image_from_plot = cv2.resize(
        image_from_plot, dsize=(output_image_width, output_image_height),
         interpolation=cv2.INTER_CUBIC)
  return image_from_plot

def determine_crop_region(
      keypoints, image_height,
      image_width):
  """Determines the region to crop the image for the model to run inference on.

  The algorithm uses the detected joints from the previous frame to estimate
  the square region that encloses the full body of the target person and
  centers at the midpoint of two hip joints. The crop size is determined by
  the distances between each joints and the center point.
  When the model is not confident with the four torso joint predictions, the
  function returns a default crop which is the full image padded to square.
  """
  target_keypoints = {}
  for joint in KEYPOINT_DICT.keys():
    target_keypoints[joint] = [
      keypoints[0, 0, KEYPOINT_DICT[joint], 0] * image_height,
      keypoints[0, 0, KEYPOINT_DICT[joint], 1] * image_width
    ]

  if torso_visible(keypoints):
    center_y = (target_keypoints['left_hip'][0] +
                target_keypoints['right_hip'][0]) / 2;
    center_x = (target_keypoints['left_hip'][1] +
                target_keypoints['right_hip'][1]) / 2;

    (max_torso_yrange, max_torso_xrange,
      max_body_yrange, max_body_xrange) = determine_torso_and_body_range(
          keypoints, target_keypoints, center_y, center_x)

    crop_length_half = np.amax(
        [max_torso_xrange * 1.9, max_torso_yrange * 1.9,
          max_body_yrange * 1.2, max_body_xrange * 1.2])

    tmp = np.array(
        [center_x, image_width - center_x, center_y, image_height - center_y])
    crop_length_half = np.amin(
        [crop_length_half, np.amax(tmp)]);

    crop_corner = [center_y - crop_length_half, center_x - crop_length_half];

    if crop_length_half > max(image_width, image_height) / 2:
      return init_crop_region(image_height, image_width)
    else:
      crop_length = crop_length_half * 2;
      return {
        'y_min': crop_corner[0] / image_height,
        'x_min': crop_corner[1] / image_width,
        'y_max': (crop_corner[0] + crop_length) / image_height,
        'x_max': (crop_corner[1] + crop_length) / image_width,
        'height': (crop_corner[0] + crop_length) / image_height -
            crop_corner[0] / image_height,
        'width': (crop_corner[1] + crop_length) / image_width -
            crop_corner[1] / image_width
      }
  else:
    return init_crop_region(image_height, image_width)

def determine_torso_and_body_range(
    keypoints, target_keypoints, center_y, center_x):
  """Calculates the maximum distance from each keypoints to the center location.

  The function returns the maximum distances from the two sets of keypoints:
  full 17 keypoints and 4 torso keypoints. The returned information will be
  used to determine the crop size. See determineCropRegion for more detail.
  """
  torso_joints = ['left_shoulder', 'right_shoulder', 'left_hip', 'right_hip']
  max_torso_yrange = 0.0
  max_torso_xrange = 0.0
  for joint in torso_joints:
    dist_y = abs(center_y - target_keypoints[joint][0])
    dist_x = abs(center_x - target_keypoints[joint][1])
    if dist_y > max_torso_yrange:
      max_torso_yrange = dist_y
    if dist_x > max_torso_xrange:
      max_torso_xrange = dist_x

  max_body_yrange = 0.0
  max_body_xrange = 0.0
  for joint in KEYPOINT_DICT.keys():
    if keypoints[0, 0, KEYPOINT_DICT[joint], 2] < MIN_CROP_KEYPOINT_SCORE:
      continue
    dist_y = abs(center_y - target_keypoints[joint][0]);
    dist_x = abs(center_x - target_keypoints[joint][1]);
    if dist_y > max_body_yrange:
      max_body_yrange = dist_y

    if dist_x > max_body_xrange:
      max_body_xrange = dist_x

  return [max_torso_yrange, max_torso_xrange, max_body_yrange, max_body_xrange]

def to_gif(images, fps):
  """Converts image sequence (4D numpy array) to gif."""
  imageio.mimsave('./animation.gif', images, fps=fps)


def progress(value, max=100):
  return HTML("""
      <progress
          value='{value}'
          max='{max}',
          style='width: 100%'
      >
          {value}
      </progress>
  """.format(value=value, max=max))

In [ ]:
# Path to the GIF file
image_path = tf.keras.utils.get_file(
    "dance_input.gif",
    "https://github.com/tensorflow/tfjs-models/raw/master/pose-detection/assets/dance_input.gif"
)

# Read and decode the GIF file
raw_data = tf.io.read_file(image_path)
gif = tf.io.decode_gif(raw_data)  # Decodes the GIF into a tensor with shape [num_frames, height, width, channels]

# Convert to a NumPy array for further processing
image = gif.numpy()

# Extract the shape
num_frames, image_height, image_width, channels = image.shape

# Debugging output
print(f"Loaded GIF with {num_frames} frames, each of size {image_height}x{image_width} with {channels} channels.")

4950608/4950608 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Loaded GIF with 42 frames, each of size 338x600 with 3 channels.


In [ ]:
# Define a to_gif function
def to_gif(images, path, fps=30):
    """
    Converts a sequence of images into a GIF file.

    Args:
        images: A NumPy array or list of images.
        path: The file path to save the GIF.
        fps: Frames per second (default is 30).
    """
    duration = 1 / fps  # Duration per frame in seconds
    with imageio.get_writer(path, mode="I", duration=duration) as writer:
        for image in images:
            writer.append_data(image)

# Generate the GIF
gif_path = "output.gif"
to_gif(output, gif_path, fps=30)  # Ensure fps is a valid number

# Display the resulting GIF
from IPython.display import Image
display(Image(filename=gif_path, format="gif"))

In [ ]:
# Load the input image (assume it's a 4D NumPy array: [num_frames, height, width, channels])
#image = tf.io.read_file(image_path)  # Load your image here
#num_frames, image_height, image_width, _ = image.shape  # Read image shape

# Initialize crop region
crop_region = init_crop_region(image_height, image_width)  # Pass appropriate arguments for initialization

# Create a list to store output frames
output_images = []

# Display progress bar
bar = display(progress(0, num_frames - 1), display_id=True)

# Process each frame
for frame_idx in range(num_frames):
    # Run inference on the current frame
    keypoints_with_scores = run_inference(
        movenet, image[frame_idx, :, :, :], crop_region,
        crop_size=[input_size, input_size]
    )

    # Visualize predictions
    output_images.append(draw_prediction_on_image(
        image[frame_idx, :, :, :].astype(np.int32),  # Convert to int32
        keypoints_with_scores, crop_region=None,
        close_figure=True, output_image_height=300
    ))

    # Update crop region for the next frame
    crop_region = determine_crop_region(
        keypoints_with_scores, image_height, image_width
    )

    # Update progress bar
    bar.update(progress(frame_idx, num_frames - 1))

# Prepare GIF visualization
output = np.stack(output_images, axis=0)  # Combine frames into a single NumPy array

# Generate the GIF
gif_path = "output.gif"
to_gif(output, gif_path)  # Call `to_gif` to save the GIF

# Display the resulting GIF
display(Image(filename=gif_path, format="gif"))

In [ ]:
# Load the input image (GIF)
image_path = tf.keras.utils.get_file(
    "dance_input.gif",
    "https://github.com/tensorflow/tfjs-models/raw/master/pose-detection/assets/dance_input.gif"
)
raw_data = tf.io.read_file(image_path)
gif = tf.io.decode_gif(raw_data)
image = gif.numpy()

# Extract image dimensions
num_frames, image_height, image_width, _ = image.shape

# Initialize crop region using a helper function
def init_crop_region(image_height, image_width):
    """
    Initialize the crop region dictionary with necessary keys.

    Args:
        image_height: Height of the image.
        image_width: Width of the image.

    Returns:
        crop_region: A dictionary containing the crop region's parameters.
    """
    return {
        "y_min": 0.0,
        "x_min": 0.0,
        "y_max": 1.0,
        "x_max": 1.0,
        "height": 1.0,  # (y_max - y_min)
        "width": 1.0   # (x_max - x_min)
    }

# Initialize crop_region
crop_region = init_crop_region(image_height, image_width)

# Initialize progress bar
def progress(current, total):
    return f"Processing frame {current}/{total}"

output_images = []
bar = display(progress(0, num_frames - 1), display_id=True)

# Processing each frame
for frame_idx in range(num_frames):
    # Perform inference on the current frame
    keypoints_with_scores = run_inference(
        movenet, image[frame_idx, :, :, :], crop_region,
        crop_size=[input_size, input_size]
    )

    # Draw predictions on the image
    output_images.append(draw_prediction_on_image(
        image[frame_idx, :, :, :].astype(np.int32),
        keypoints_with_scores, crop_region=None,
        close_figure=True, output_image_height=300
    ))

    # Update crop region based on keypoints
    crop_region = determine_crop_region(
        keypoints_with_scores, image_height, image_width
    )

    # Update progress bar
    bar.update(progress(frame_idx, num_frames - 1))

# Prepare GIF visualization
output = np.stack(output_images, axis=0)

# Function to save images as a GIF
def to_gif(images, path, fps=30):
    duration = 1 / fps
    with imageio.get_writer(path, mode="I", duration=duration) as writer:
        for img in images:
            writer.append_data(img)

# Save the output as a GIF
output_gif_path = "output.gif"
to_gif(output, output_gif_path, fps=30)

# Display the resulting GIF
display(Image(filename=output_gif_path, format="gif"))




**Part 3- Using webcam video**

In [ ]:
# Step 1: Install necessary libraries
!pip install torch torchvision torchaudio
!pip install ultralytics
!pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.3/887.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.7 MB/s eta 0:00:00


In [ ]:
# Install YOLOv8
!pip install ultralytics opencv-python-headless

In [ ]:
# Load Required libraries
from ultralytics import YOLO
import cv2
import numpy as np
import torch
import requests
import ipywidgets as widgets
from IPython.display import display, Javascript
from google.colab.output import eval_js
import base64
from base64 import b64decode
from io import BytesIO
from PIL import Image
from ultralytics import YOLO

In [ ]:
#b- Working on webcam
#Function to activate your laptop cam. This function is complete and you don't need to do anything.
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
# Step 2: Load pre-trained YOLOv8 model
model =  YOLO('yolov8n.pt')  # Add a code here to Load YOLOv8 nano model for better performance

100%|██████████| 6.25M/6.25M [00:00<00:00, 78.3MB/s]


In [ ]:
# Step 3: Function to detect objects using YOLOv8
def detect_objects(frame):
    results = model(frame)
    annotated_frame = results[0].plot()
    return annotated_frame

In [ ]:
# Step 4: JavaScript to capture video from webcam
def capture_video():
    display(Javascript('''
        async function captureVideo() {
            const video = document.createElement('video');
            const canvas = document.createElement('canvas');
            const context = canvas.getContext('2d');
            document.body.appendChild(video);
            video.style.display = 'none';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            video.srcObject = stream;
            await new Promise((resolve) => video.onloadedmetadata = resolve);
            video.play();

            const [track] = stream.getVideoTracks();
            const settings = track.getSettings();
            canvas.width = settings.width;
            canvas.height = settings.height;

            async function stopVideo() {
                stream.getTracks().forEach(track => track.stop());
                video.remove();
                canvas.remove();
            }

            window.stopVideo = stopVideo;

            while (true) {
                context.drawImage(video, 0, 0, canvas.width, canvas.height);
                const imageData = canvas.toDataURL('image/jpeg', 0.8);
                const response = await google.colab.kernel.invokeFunction('notebook.getFrame', [imageData], {});
                if (response.data.result === 'quit') break;
                await new Promise((resolve) => setTimeout(resolve, 100));
            }

            stream.getTracks().forEach(track => track.stop());
            video.remove();
            canvas.remove();
        }
        captureVideo();
    '''))

In [ ]:
# Step 5: Python callback to process frames
def get_frame(image_data):
    try:
        image_data = image_data.split(",")[1]
        image_bytes = base64.b64decode(image_data)
        image = Image.open(BytesIO(image_bytes))
        frame = np.array(image)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_detected = detect_objects(frame_rgb)
        _, img_encoded = cv2.imencode('.jpg', frame_detected)
        image_widget.value = img_encoded.tobytes()
        return {'result': 'continue'}
    except Exception as e:
        print(f"Error processing frame: {e}")
        return {'result': 'error'}


In [ ]:
# Register the Python callback function
from google.colab import output
output.register_callback('notebook.getFrame', get_frame)

# Step 6: Widget to display the video stream
image_widget = widgets.Image(format='jpeg')
display(image_widget)


Image(value=b'', format='jpeg')

In [ ]:
# Step 7: Stop button functionality otherwise constantly the video will be shown
def stop_video(change):
    eval_js('window.stopVideo()')
    return {'result': 'quit'}

In [ ]:
# Now call the functions and see the object detection in video

# Create and display a stop button
stop_button = widgets.Button(description="Stop Video")
stop_button.on_click(stop_video)
display(stop_button)

# Start capturing video
capture_video()

Button(description='Stop Video', style=ButtonStyle())

<IPython.core.display.Javascript object>

Streaming output truncated to the last 5000 lines.
0: 480x640 1 person, 183.3ms
Speed: 3.0ms preprocess, 183.3ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 200.0ms
Speed: 2.5ms preprocess, 200.0ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 206.2ms
Speed: 3.7ms preprocess, 206.2ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 192.6ms
Speed: 3.1ms preprocess, 192.6ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 190.1ms
Speed: 3.6ms preprocess, 190.1ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 198.1ms
Speed: 3.3ms preprocess, 198.1ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 202.3ms
Speed: 2.4ms preprocess, 202.3ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 216.5ms
Speed: 2.6ms preprocess,

Extra credit: Instead of taking photos, design a code that can detect the objects in real-time
while you are seeing the video on Notebook.

The code is shown below:  the main change is to continuously capture video frames from the webcam, processing them for object detection, and then displaying the resulting frames in real-time.

In [ ]:
# Load YOLOv8 model (You can use any YOLOv8 model)
model = YOLO('yolov8n.pt')  # Example using the small YOLOv8 model

# Function to detect objects using YOLOv8
def detect_objects(frame):
    results = model(frame)
    annotated_frame = results[0].plot()  # Draws bounding boxes on the image
    return annotated_frame

# JavaScript to capture video from the webcam and send it to Python callback
def capture_video():
    display(Javascript('''
        async function captureVideo() {
            const video = document.createElement('video');
            const canvas = document.createElement('canvas');
            const context = canvas.getContext('2d');
            document.body.appendChild(video);
            video.style.display = 'none';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            video.srcObject = stream;
            await new Promise((resolve) => video.onloadedmetadata = resolve);
            video.play();

            const [track] = stream.getVideoTracks();
            const settings = track.getSettings();
            canvas.width = settings.width;
            canvas.height = settings.height;

            async function stopVideo() {
                stream.getTracks().forEach(track => track.stop());
                video.remove();
                canvas.remove();
            }

            window.stopVideo = stopVideo;

            while (true) {
                context.drawImage(video, 0, 0, canvas.width, canvas.height);
                const imageData = canvas.toDataURL('image/jpeg', 0.8);
                const response = await google.colab.kernel.invokeFunction('notebook.getFrame', [imageData], {});
                if (response.data.result === 'quit') break;
                await new Promise((resolve) => setTimeout(resolve, 100));
            }

            stream.getTracks().forEach(track => track.stop());
            video.remove();
            canvas.remove();
        }
        captureVideo();
    '''))

# Python callback to process frames and run YOLOv8 object detection
def get_frame(image_data):
    try:
        # Decode the base64 image data
        image_data = image_data.split(",")[1]
        image_bytes = base64.b64decode(image_data)
        image = Image.open(BytesIO(image_bytes))
        frame = np.array(image)

        # Convert the frame to RGB (OpenCV uses BGR by default)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Perform object detection using YOLOv8
        frame_detected = detect_objects(frame_rgb)

        # Encode the processed frame to JPEG for display
        _, img_encoded = cv2.imencode('.jpg', frame_detected)

        # Update the widget with the processed frame
        image_widget.value = img_encoded.tobytes()

        return {'result': 'continue'}
    except Exception as e:
        print(f"Error processing frame: {e}")
        return {'result': 'error'}

# Register the Python callback function to process frames from JavaScript
from google.colab import output
output.register_callback('notebook.getFrame', get_frame)

# Widget to display the video stream with object detection
image_widget = widgets.Image(format='jpeg')
display(image_widget)

# Stop button functionality to stop video capture
def stop_video(change):
    eval_js('window.stopVideo()')
    return {'result': 'quit'}

# Create and display a stop button
stop_button = widgets.Button(description="Stop Video")
stop_button.on_click(stop_video)
display(stop_button)

# Start capturing video
capture_video()

Image(value=b'', format='jpeg')

Button(description='Stop Video', style=ButtonStyle())

<IPython.core.display.Javascript object>

Streaming output truncated to the last 5000 lines.
0: 480x640 1 person, 290.3ms
Speed: 2.5ms preprocess, 290.3ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 181.0ms
Speed: 2.6ms preprocess, 181.0ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 189.8ms
Speed: 2.5ms preprocess, 189.8ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 179.5ms
Speed: 3.3ms preprocess, 179.5ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 178.5ms
Speed: 2.6ms preprocess, 178.5ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 206.8ms
Speed: 3.9ms preprocess, 206.8ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 200.3ms
Speed: 3.0ms preprocess, 200.3ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 199.0ms
Speed: 3.6ms preprocess,